# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def step_one(*, k):
    from TaskLLM import TaskLLM
    from PersonalizedCitation import train_loader, dev_loader
    from ProgressPrinter import ProgressPrinter
    from peft import IA3Config, TaskType, prepare_model_for_kbit_training
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    import warnings
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=1)
    dev = dev_loader(batch_size=1)

    t5 = prepare_model_for_kbit_training(T5ForConditionalGeneration.from_pretrained('google/flan-t5-xxl', load_in_8bit=True))
    taskllm_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM)
    t5.add_adapter(taskllm_config, "taskllm")
    t5.enable_adapters()

    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")

    with ProgressPrinter('iter', f'{k} loss', f'{k} acc', f'{k} acc (dev)') as printer, warnings.catch_warnings():
        warnings.filterwarnings("ignore", message=".*MatMul8bitLt.*")
        warnings.filterwarnings("ignore", message=".*If you want to save 8-bit models.*")
        
        for iteration in range(2):
            for istrain, (examples, labels) in interleave(train, dev):
                if iteration == 0 or not istrain:
                    with torch.no_grad():
                        prompts = []
                        target = []
        
                        for ex, label in zip(examples, labels):
                            embeddings = train.embed( [ ex['ref1'], ex['ref2'] ] + 
                                                      [ v['title'] 
                                                       for v in ex['profile']
                                                       if v['title'] != ex['title'] 
                                                     ])
                            scores = torch.max(embeddings[[0,1],:] @ embeddings[2:,:].T, dim=0).values
                            index = torch.topk(scores, dim=0, k=k).indices.to('cpu')
                            titles = [ f'{ex["profile"][ind]["title"]}' for ind in index.tolist() ]
                            concat_titles = ' and '.join([f'"{v}"' for v in titles])
                            prompt = train.append_to_title(ex, concat_titles)
                            prompts.append(prompt)
                            target.append(int(label == train.choices[1]))

                        target = torch.Tensor(target).long().to(device)
                        acc = (taskllm.predict(prompts, augment=train.swap_refs).argmax(dim=1) == target).float().mean().item()
    
                    loss = taskllm.learn(prompts, target, augment=train.swap_refs) if istrain else None
                    printer.addobs(iteration, loss, acc if istrain else None, acc if not istrain else None)

            printer.print()
            printer.autoprint = False
            if iteration == 0:
                taskllm.save_pretrained(f'User_keq{k}_t5xxl_step1')

from Fork import SubProcess
with SubProcess() as process: process.parent or step_one(k=4)

n                  iter       since      4 loss       since       4 acc       since 4 acc (dev)       since      dt (s)
1                     0           0       0.607       0.607           1           1           0           0        3.46
2                     0           0       0.899        1.19         0.5           0           0           0        6.02
4                     0           0        0.76       0.484       0.667           1           1           1        9.25
8                     0           0       0.717       0.684       0.571         0.5           1           0        19.5
16                    0           0       0.751       0.791       0.462       0.333       0.667         0.5        35.9
32                    0           0       0.706       0.661       0.538       0.615       0.833           1        70.2
64                    0           0       0.692       0.677       0.569         0.6       0.923           1         139
128                   0           0     

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def learn_ranker(*, max_iteration, k):
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedCitation import train_loader, dev_loader
    from ProgressPrinter import ProgressPrinter
    from SimpleRegret import SimpleRegretGreedyDoubleSampler
    from peft import IA3Config, TaskType, prepare_model_for_kbit_training
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    import warnings
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=1, double_data=True)
    dev = dev_loader(batch_size=1)

    t5 = prepare_model_for_kbit_training(T5ForConditionalGeneration.from_pretrained('google/flan-t5-xxl', load_in_8bit=True))
    t5.load_adapter('User_keq4_t5xxl_step1', 'taskllm')

    rhat_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM)
    t5.add_adapter(rhat_config, "rhat")
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat")
    
    def reward_augment(prompts):
        import re
        return [ re.sub(r'Ref1: (.*)\nRef2: (.*)\nExtra:',
                        r'Ref1: \2\nRef2: \1\nExtra:',
                        z)
                 for z in prompts ]

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(ex, nsamples):
        embeddings = train.embed( [ ex['ref1'], ex['ref2'] ] + 
                                  [ v['title'] 
                                   for v in ex['profile']
                                   if v['title'] != ex['title'] 
                                 ])
        scores = torch.max(embeddings[[0,1],:] @ embeddings[2:,:].T, dim=0).values
        temperature = scores[0].item() - scores[4].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape)
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=k).indices, sorted=False, dim=0).to('cpu')

    with ProgressPrinter('iter', f'{k} loss', f'{k} acc', f'{k} acc (dev)') as printer, warnings.catch_warnings():
        warnings.filterwarnings("ignore", message=".*MatMul8bitLt.*")
        warnings.filterwarnings("ignore", message=".*If you want to save 8-bit models.*")
        
        for iteration in range(max_iteration):
            for istrain, (examples, labels) in interleave(train, dev):
                if iteration + 1 < max_iteration or not istrain:
                    for ex, label in zip(examples, labels):
                        greedyrewards = []
                        allloss = []
                        with torch.no_grad():
                            randos = randomized_similarity(ex, 64)
                            
                            rhatprompts = []
                            prompts = []
                            for rando in randos:
                                titles = [ f'{ex["profile"][ind]["title"]}' for ind in rando ]
                                concat_titles = ' and '.join([f'"{v}"' for v in titles])
                                prompt = train.append_to_title(ex, concat_titles)
                                prompts.append(prompt)
                                rhatprompt = f"Title: {ex['title']}\nRef1: {ex['ref1']}\nRef2: {ex['ref2']}\n" + '\n'.join(
                                           [ f"Extra: {t}" for t in titles ])
                                rhatprompts.append(rhatprompt)

                            rhats = rewardpredictor.predict(rhatprompts, augment=reward_augment)
                            if len(rhats) > 1:
                                explore, exploit = SimpleRegretGreedyDoubleSampler(rhats.view(1, -1), gamma=10)
                                actionind = [exploit.item(), explore.item()]
                            else:
                                actionind = [0]

                            guesses = taskllm.predict([ prompts[a] for a in actionind ], augment=train.swap_refs).argmax(dim=1)
                            target = int(label == train.choices[1])
                            rewards = (guesses == target).float().unsqueeze(1)
                            greedyreward = rewards[0, 0].item()
                            greedyrewards.append(greedyreward)
                            
                        loss = rewardpredictor.learn([ rhatprompts[a] for a in actionind ], rewards, augment=reward_augment) if istrain else None
                        allloss.append(loss)

                    greedyacc = torch.Tensor(greedyrewards).float().mean().item()
                    predloss = torch.Tensor(allloss).mean().item() if istrain else None

                    printer.addobs(iteration, predloss, greedyacc if istrain else None, greedyacc if not istrain else None)

            printer.print()
            printer.autoprint = False
            if iteration + 1 < max_iteration:
                rewardpredictor.save_pretrained(f'User_keq{k}_t5xxl_step2_iter{iteration}')

from Fork import SubProcess
with SubProcess() as process: process.parent or learn_ranker(k=4, max_iteration=8)